In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /home/edu/.local/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp06m9zuma
  JVM stdout: /tmp/tmp06m9zuma/h2o_edu_started_from_python.out
  JVM stderr: /tmp/tmp06m9zuma/h2o_edu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_edu_c055bc
H2O cluster total nodes:,1
H2O cluster free memory:,1.887 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [64]:
train_high = h2o.import_file('../cleaned-data/03-train_high.csv')
train_low = h2o.import_file('../cleaned-data/03-train_low.csv')
test_high = h2o.import_file('../cleaned-data/03-test_high.csv')
test_low = h2o.import_file('../cleaned-data/03-test_low.csv')                    

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [65]:
train_high_noid = train_high.drop(index=[0], axis=0)
train_low_noid = train_low.drop(index=[0], axis=0)
test_high_noid = test_high.drop(index=[0], axis=0)
test_low_noid = test_low.drop(index=[0], axis=0)

In [66]:
aml = H2OAutoML(max_runtime_secs = 120, project_name = "diamonds", keep_cross_validation_models=True)
aml.train(y = 'price', training_frame = train_high_noid)

AutoML progress: |
22:52:22.609: New models will be added to existing leaderboard diamonds@@price (leaderboard frame=null) with already 23 models.
22:52:33.640: StackedEnsemble_BestOfFamily_AutoML_20200229_225222 [StackedEnsemble best (built using top model from each algorithm type)] failed: water.exceptions.H2OIllegalArgumentException: Base models are inconsistent: they use different size (number of rows) training frames. Found: 1614 (StackedEnsemble) and 1615 (model GBM_3_AutoML_20200229_223736).
22:52:34.644: StackedEnsemble_AllModels_AutoML_20200229_225222 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Base models are inconsistent: they use different size (number of rows) training frames. Found: 1614 (StackedEnsemble) and 1615 (model GBM_3_AutoML_20200229_223736).
22:54:24.147: New models will be added to existing leaderboard diamonds@@price (leaderboard frame=null) with already 25 models.
22:56:26.678: StackedEnsemble_Be

In [67]:
pred_leader = aml.leader.predict(test_high_noid)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [68]:
pred_df = pred_leader.as_data_frame()
pred_df = pred_df.merge(train_high['C1'].as_data_frame(), left_index=True, right_index=True)
pred_df = pred_df.rename(columns={'C1':'id', 'predict':'price'})

In [69]:
pred_df[['id','price']].to_csv('../predictions/07-Auto-ML-v4-high-carat.csv', index=False)

In [70]:
pred_df

,price,id
0,10328.174805,8
1,11253.188477,34
2,12680.707031,42
3,11840.192383,72
4,13744.352539,82
...,...,...
1609,9414.047852,40155
1610,16612.927734,40225
1611,16552.197266,40274
1612,14356.041992,40276


In [71]:
aml.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20200229_225928,184204,429.19,184204,228.468,0.0920145
StackedEnsemble_BestOfFamily_AutoML_20200229_223736,2.85853e+06,1690.72,2.85853e+06,1323.73,0.121441
StackedEnsemble_AllModels_AutoML_20200229_223736,2.86208e+06,1691.77,2.86208e+06,1323.3,0.12152
XGBoost_3_AutoML_20200229_230602,2.88205e+06,1697.66,2.88205e+06,1346.21,0.121767
XGBoost_3_AutoML_20200229_225424,2.88224e+06,1697.72,2.88224e+06,1343.36,0.121975
XGBoost_3_AutoML_20200229_223736,2.88376e+06,1698.16,2.88376e+06,1342.08,0.122036
GBM_3_AutoML_20200229_223736,2.89222e+06,1700.65,2.89222e+06,1336.45,0.122088
XGBoost_1_AutoML_20200229_223736,2.89357e+06,1701.05,2.89357e+06,1344.59,0.122011
GBM_2_AutoML_20200229_225424,2.8946e+06,1701.35,2.8946e+06,1334.66,0.122537
GBM_grid__1_AutoML_20200229_225424_model_3,2.89533e+06,1701.57,2.89533e+06,1341.56,0.122963


In [78]:
aml_low = H2OAutoML(max_runtime_secs = 120, project_name = "diamonds", keep_cross_validation_models=True)
aml_low.train(y = 'price', training_frame = train_low_noid)

AutoML progress: |
22:52:22.609: New models will be added to existing leaderboard diamonds@@price (leaderboard frame=null) with already 23 models.
22:52:33.640: StackedEnsemble_BestOfFamily_AutoML_20200229_225222 [StackedEnsemble best (built using top model from each algorithm type)] failed: water.exceptions.H2OIllegalArgumentException: Base models are inconsistent: they use different size (number of rows) training frames. Found: 1614 (StackedEnsemble) and 1615 (model GBM_3_AutoML_20200229_223736).
22:52:34.644: StackedEnsemble_AllModels_AutoML_20200229_225222 [StackedEnsemble all (built using all AutoML models)] failed: water.exceptions.H2OIllegalArgumentException: Base models are inconsistent: they use different size (number of rows) training frames. Found: 1614 (StackedEnsemble) and 1615 (model GBM_3_AutoML_20200229_223736).
22:54:24.147: New models will be added to existing leaderboard diamonds@@price (leaderboard frame=null) with already 25 models.
22:56:26.678: StackedEnsemble_Be

In [79]:
aml_low.leaderboard

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20200229_231222,183571,428.452,183571,228.407,0.0919347
XGBoost_1_AutoML_20200229_225928,184204,429.19,184204,228.468,0.0920145
XGBoost_1_AutoML_20200229_230803,184950,430.058,184950,229.575,0.092267
XGBoost_2_AutoML_20200229_231222,579427,761.201,579427,438.987,0.149345
StackedEnsemble_BestOfFamily_AutoML_20200229_223736,2.85853e+06,1690.72,2.85853e+06,1323.73,0.121441
StackedEnsemble_AllModels_AutoML_20200229_223736,2.86208e+06,1691.77,2.86208e+06,1323.3,0.12152
XGBoost_3_AutoML_20200229_230602,2.88205e+06,1697.66,2.88205e+06,1346.21,0.121767
XGBoost_3_AutoML_20200229_225424,2.88224e+06,1697.72,2.88224e+06,1343.36,0.121975
XGBoost_3_AutoML_20200229_223736,2.88376e+06,1698.16,2.88376e+06,1342.08,0.122036
GBM_3_AutoML_20200229_223736,2.89222e+06,1700.65,2.89222e+06,1336.45,0.122088


In [80]:
pred_low = aml.leader.predict(test_low)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [84]:
pred_low

,price,id
0,10328.174805,0
1,11253.188477,1
2,12680.707031,2
3,11840.192383,3
4,13744.352539,4
...,...,...
1609,9414.047852,1683
1610,16612.927734,1684
1611,16552.197266,1685
1612,14356.041992,1686


In [85]:
#pred_low = pred_low.as_data_frame()
pred_low = pred_low.merge(test_low['C1'].as_data_frame(), left_index=True, right_index=True)
pred_low = pred_low.rename(columns={'C1':'id', 'predict':'price'})
pred_low[['id','price']].to_csv('../predictions/07-Auto-ML-v4-low-carat.csv', index=False)

In [86]:
display(pred_df, pred_low)

,price,id
0,10328.174805,8
1,11253.188477,34
2,12680.707031,42
3,11840.192383,72
4,13744.352539,82
...,...,...
1609,9414.047852,40155
1610,16612.927734,40225
1611,16552.197266,40274
1612,14356.041992,40276


,price,id,id
0,10328.174805,0,0
1,11253.188477,1,1
2,12680.707031,2,2
3,11840.192383,3,3
4,13744.352539,4,4
...,...,...,...
1609,9414.047852,1683,1683
1610,16612.927734,1684,1684
1611,16552.197266,1685,1685
1612,14356.041992,1686,1686


In [ ]:
test_pred = pd.concat([pred_df, pred_low],ignore_index=True)
test_pred.head()

In [ ]:
test_pred.to_csv('..')